In [74]:
from dotenv import load_dotenv

load_dotenv()

True

# ReWOO

In [75]:
from typing import List
from typing_extensions import TypedDict


class ReWOO(TypedDict):
    task: str
    plan_string: str
    steps: List
    results: dict
    result: str


# Planner

In [76]:
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(model="gpt-4o-mini")


In [77]:
prompt = """For the following task, make plans that can solve the problem step by step. For each plan, indicate \
which external tool together with tool input to retrieve evidence. You can store the evidence into a \
variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:
(1) Google[input]: Worker that searches results from Google. Useful when you need to find short
and succinct answers about a specific topic. The input should be a search query.
(2) LLM[input]: A pretrained LLM like yourself. Useful when you need to act with general
world knowledge and common sense. Prioritize it when you are confident in solving the problem
yourself. Input can be any instruction.

For example,
Task: Thomas, Toby, and Rebecca worked a total of 157 hours in one week. Thomas worked x
hours. Toby worked 10 hours less than twice what Thomas worked, and Rebecca worked 8 hours
less than Toby. How many hours did Rebecca work?
Plan: Given Thomas worked x hours, translate the problem into algebraic expressions and solve
with Wolfram Alpha. #E1 = WolframAlpha[Solve x + (2x − 10) + ((2x − 10) − 8) = 157]
Plan: Find out the number of hours Thomas worked. #E2 = LLM[What is x, given #E1]
Plan: Calculate the number of hours Rebecca worked. #E3 = Calculator[(2 ∗ #E2 − 10) − 8]

Begin! 
Describe your plans with rich details. Each Plan should be followed by only one #E.

Task: {task}"""


In [78]:
task = "what is the exact hometown of the 2024 mens australian open winner"


In [79]:
result = model.invoke(prompt.format(task=task))


In [24]:
print(result.content)


Plan: The first step is to identify the winner of the 2024 Men's Australian Open. Since the tournament occurs annually in January, the winner should be available from recent news. I will search for the winner using Google.
#E1 = Google["2024 Men's Australian Open winner"]

Plan: Once I have identified the winner, I need to find his exact hometown. This can be done by searching for the hometown of the winner using Google.
#E2 = Google["Exact hometown of #E1"]


Planner Node

To connect the planner to our graph, we will create a get_plan node that accepts the ReWOO state and returns with a state update for the steps and plan_string fields.

In [80]:
import re

from langchain_core.prompts import ChatPromptTemplate

# Regex to match expressions of the form E#... = ...[...]
regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]"
prompt_template = ChatPromptTemplate.from_messages([("user", prompt)])
planner = prompt_template | model


def get_plan(state: ReWOO):
    task = state["task"]
    result = planner.invoke({"task": task})
    # Find all matches in the sample text
    matches = re.findall(regex_pattern, result.content)
    return {"steps": matches, "plan_string": result.content}


# Executor

In [81]:
from langchain_core.tools import tool
from web_search import process_search_results, async_get_page_content


@tool
async def search(query):
    """
    tool for google search internet
    Args:
        query: str
    Output:
        List[str, any]
    """
    content_list = []
    urls = await process_search_results(input=query)
    # return urls
    for url in urls:
        content = await async_get_page_content(url)
        content_list.append(content)
    return content_list

In [28]:
result = await search.ainvoke("hiện tại là mấy giờ ở vn")
result

2024-12-05 16:27:16.841 | INFO     | web_search:process_search_results:22 - Before generate
2024-12-05 16:27:16.843 | INFO     | web_search:process_search_results:23 - Use Web
2024-12-05 16:27:16.844 | INFO     | web_search:process_search_results:25 - English question: ['hiện tại là mấy giờ ở vn']
2024-12-05 16:27:16.845 | INFO     | web_search:process_search_results:27 - After generate
2024-12-05 16:27:16.846 | INFO     | web_search:process_search_results:31 - Before get_search_results


Queries: ['hiện tại là mấy giờ ở vn']


2024-12-05 16:27:17.976 | INFO     | web_search:fetch_and_parse:150 - Fetch url: https://onlinealarmkur.com/clock/en/
2024-12-05 16:27:18.094 | INFO     | web_search:fetch_and_parse:150 - Fetch url: https://vi.thetimenow.com/vietnam
2024-12-05 16:27:19.388 | INFO     | web_search:fetch_and_parse:150 - Fetch url: https://time.is/vi/Hanoi


['Online Clock - Digital and Analog\nMenu\nOnline Alarm Clock\nOnline Timer\nOnline Countdown\nOnline Stopwatch\nOnline Clock\nWorld Clock\nDate Calculator\nHours Calculator\nWeek Number\nCalendar 2025\nOnline Clock\nThursday, November 28, 2024\n12:00:00 AM\n1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n12\nDigital Clock\nAnalog Clock\nWhat Is the Online Clock?\nWhen you want to know, "What time is it?", you can use the online clock to see the current time in hours, minutes, and seconds.\nTo see the time in different countries and timezones around the world, check the\nworld clock\n.\nHow Do I Use the Online Clock?\nThe online clock doesn\'t let you track time passing. You can use the\nonline stopwatch\n,\nonline timer\n, or\nonline alarm clock\nfor these tasks. All you need to do is view the clock with the date.\nHere\'s how to use the online clock for different tasks:\nTo view the current time digitally, click\nDigital Clock\n.\nTo know, "What time is it?" in analog, tap\nAnalog Clock\n.\nTo ma

In [82]:
def _get_current_task(state: ReWOO):
    if "results" not in state or state["results"] is None:
        return 1
    if len(state["results"]) == len(state["steps"]):
        return None
    else:
        return len(state["results"]) + 1


async def tool_execution(state: ReWOO):
    """Worker node that executes the tools of a given plan."""
    _step = _get_current_task(state)
    _, step_name, tool, tool_input = state["steps"][_step - 1]
    _results = (state["results"] or {}) if "results" in state else {}
    for k, v in _results.items():
        tool_input = tool_input.replace(k, v)
    if tool == "Google":
        result = await search.ainvoke(tool_input)
    elif tool == "LLM":
        result = await model.ainvoke(tool_input)
    else:
        raise ValueError
    _results[step_name] = str(result)
    return {"results": _results}


# Solver

In [83]:
solve_prompt = """Solve the following task or problem. To solve the problem, we have made step-by-step Plan and \
retrieved corresponding Evidence to each Plan. Use them with caution since long evidence might \
contain irrelevant information.

{plan}

Now solve the question or task according to provided Evidence above. Respond with the answer
directly with no extra words.

Task: {task}
Response:"""


async def solve(state: ReWOO):
    plan = ""
    for _plan, step_name, tool, tool_input in state["steps"]:
        _results = (state["results"] or {}) if "results" in state else {}
        for k, v in _results.items():
            tool_input = tool_input.replace(k, v)
            step_name = step_name.replace(k, v)
        plan += f"Plan: {_plan}\n{step_name} = {tool}[{tool_input}]"
    prompt = solve_prompt.format(plan=plan, task=state["task"])
    result = await model.ainvoke(prompt)
    return {"result": result.content}


# Define Graph

In [84]:
def _route(state):
    _step = _get_current_task(state)
    if _step is None:
        # We have executed all tasks
        return "solve"
    else:
        # We are still executing tasks, loop back to the "tool" node
        return "tool"


In [85]:
from langgraph.graph import END, StateGraph, START

graph = StateGraph(ReWOO)
graph.add_node("plan", get_plan)
graph.add_node("tool", tool_execution)
graph.add_node("solve", solve)
graph.add_edge("plan", "tool")
graph.add_edge("solve", END)
graph.add_conditional_edges("tool", _route)
graph.add_edge(START, "plan")

app = graph.compile()


In [86]:
async for s in app.astream({"task": task}):
    print(s)
    print("---")


{'plan': {'plan_string': 'To solve the task of finding the exact hometown of the 2024 Men\'s Australian Open winner, I will create a series of plans to gather the necessary information.\n\n### Plan 1: Search for the winner of the 2024 Men\'s Australian Open\nTo start, I need to identify the winner of the 2024 Men\'s Australian Open. I\'ll use Google to search for this information.\n- **#E1** = Google["2024 Men\'s Australian Open winner"]\n\n### Plan 2: Find the hometown of the identified winner\nOnce I have the name of the winner from the first plan, I will then search for their hometown. I will again use Google for a specific search related to the winner\'s background.\n- **#E2** = Google["hometown of #E1"]\n\n### Plan 3: Verify the accuracy of the hometown information\nTo ensure that the information about the winner\'s hometown is accurate, I will use an LLM to contextualize and confirm the details gathered in the previous plans.\n- **#E3** = LLM["Confirm the hometown of the winner #

IndexError: list index out of range

In [54]:
# Print out the final result
print(s["solve"]["result"])


Jannik Sinner's exact hometown is San Candido, Italy.
